## **Geração de Instâncias**

In [1]:
import pickle

#### Escolha <ins>**uma**</ins> das opções:

**1 - Instâncias fixas:** a célula abaixo pega as instâncias que geramos, executamos e relatamos do arquivo `fixed_instances.pkl`, as salvando na lista de instâncias `instances`.

In [2]:
# Carregar instâncias em 'fixed_instances.pkl'
with open("fixed_instances.pkl", "rb") as fp:
    instances = pickle.load(fp)

**2 - Novas instâncias:** a célula abaixo gera novas instâncias diferentes da que testamos, as salvando no arquivo `instances.pkl` e na lista de instâncias `instances`.

In [16]:
from numpy.random import randint

instances = []

# Uma instância será criada p/ cada nº de clientes abaixo
num_of_clients_list = (50, 100, 150, 200, 250, 300, 350, 400, 450, 500)

for J in num_of_clients_list:
    
    F = randint(J, 2*J) # Fábricas
    L = randint(5, 10) # Máquinas
    M = randint(5, 10) # Tipos de matéria-prima
    P = randint(5, 10) # Tipos de papel
    D = randint(10, 20, size=(J, P)) # Demanda do cliente 'j' em papel 'p'
    r = randint(1, 5, size=(M, P, L)) # Qtd. de 'm' p/ prod. 1 'p' na máq. 'l'
    R = randint(800, 1000, size=(M, F)) # Qtd. de 'm' em cada fábrica 'f'
    C = randint(80, 100, size=(L, F)) # Cap. de prod. da máq. 'l' na fábrica 'f'
    p = randint(10, 100, size=(P, L, F)) # Custo de prod. 'p' usando máq. 'l' na fábrica 'f'
    t = randint(10, 20, size=(P, F, J)) # Custo de transp. 'p' da fábrica 'f' ao cliente 'j'
    
    # Salvar cada instância em um dicionário
    instances.append({
        "J": J,
        "F": F,
        "L": L,
        "M": M,
        "P": P,
        "D": D,
        "r": r,
        "R": R,
        "C": C,
        "p": p,
        "t": t,
    })

# Salvar instâncias em 'instances.pkl' 
with open('instances.pkl', 'wb') as output:
    pickle.dump(instances, output)
    

## **Definição do Modelo**

In [3]:
from gurobipy import GRB
import gurobipy as gp

def create_model(inst, relaxed = False):
    '''
    Função que define o modelo para o Problema de Produção de Papel,
    seja ele o modelo de PLI (se 'relaxed' for 'False') ou o modelo 
    relaxado de PL (se 'relaxed' for 'True').
    '''
            
    # Inicializar modelo vazio
    model = gp.Model('paper-production')
    
    # Limitar tempo de execução em 30 minutos
    model.setParam('TimeLimit', 1800) 
    
    # O tipo das variáveis é inteiro por default e contínuo para o modelo
    # com as restrições de integralidade relaxadas
    vtype = (GRB.INTEGER if not relaxed else GRB.CONTINUOUS)

    # Variáveis de decisão:
    # - x[p, l, f]: qtd. de papel 'p' produzida na máq. 'l' da fábrica 'f';
    x = model.addVars(inst['P'], inst['L'], inst['F'], vtype=vtype)
    # - y[p, f, j]: qtd. de papel 'p' transp. da fábrica 'f' até o cliente 'j';
    y = model.addVars(inst['P'], inst['F'], inst['J'], vtype=vtype)

    # Objetivo:
    model.setObjective(
        # minimizar custo de produção...
        gp.quicksum(inst['p'][p,l,f] * x[p,l,f] for p in range(inst['P']) 
                                                for l in range(inst['L']) 
                                                for f in range(inst['F'])) +
        # e o de transporte
        gp.quicksum(inst['t'][p,f,j] * y[p,f,j] for p in range(inst['P']) 
                                                for f in range(inst['F']) 
                                                for j in range(inst['J']))
    )        

    # Restrições:
    # - não transportar mais do que é produzido;
    model.addConstrs((x.sum(p, '*', f) >= y.sum(p, f, '*') for p in range(inst['P'])
                                                           for f in range(inst['F'])), name='link_xy')
    # - atender demanda dos clientes;
    model.addConstrs((y.sum(p, '*', j) >= inst['D'][j, p] for p in range(inst['P']) 
                                                          for j in range(inst['J'])), name='dem')
    # - não ultrapassar capacidade das máquinas;
    model.addConstrs((x.sum('*', l, f) <= inst['C'][l, f] for l in range(inst['L']) 
                                                          for f in range(inst['F'])), name='cap')
    # - não exceder qtd. disponível de matéria-prima;
    model.addConstrs(
        (
            # a qtd. de matéria usada na máq. 'm' da fáb. 'f' não pode exceder R[m,f]...
            gp.quicksum(inst['r'][m, p, l] * x[p, l, f] for p in range(inst['P']) 
                                                        for l in range(inst['L'])) 
            <= inst['R'][m, f]
            # ... para toda máquina e fábrica
            for m in range(inst['M'])
            for f in range(inst['F']) 
        ), name='mp'
    )
    
    return model
    

## **Testes**

Os testes são executados sob as instâncias salvas na lista `instances`, que depende de qual célula foi executada no início desse Notebook.

#### **Modelo PLI**: sem as restrições de integralidade relaxadas.

In [59]:
for inst in instances:
    print('\nSOL. DA INSTÂNCIA COM ' + str(inst['J']) + ' CLIENTES:\n')
    
    # Criar modelo
    model = create_model(inst)
    
    # Computar solução ótima
    model.optimize()
    
    # Relatar outros dados
    print(
        '\nTempo de execução: ' + 
        str(model.runTime) + 
        '; Número de variáveis: ' + 
        str(model.numVars) + 
        '\n'
    )
    


SOL. DA INSTÂNCIA COM 50 CLIENTES:

Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 2122 rows, 36736 columns and 101024 nonzeros
Model fingerprint: 0x914ce4ec
Variable types: 0 continuous, 36736 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 416139.00000
Presolve time: 0.09s
Presolved: 2122 rows, 36736 columns, 101024 nonzeros
Variable types: 0 continuous, 36736 integer (0 binary)

Root relaxation: objective 1.222440e+05, 3332 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                    122244.00000    0.00000   100%     -    0s
     0     0 122244.000

#### **Modelo PL**: com as restrições de integralidade relaxadas.

In [4]:
for inst in instances:
    print('\nSOL. DA INSTÂNCIA COM ' + str(inst['J']) + ' CLIENTES:\n')
    
    # Criar modelo relaxado
    model = create_model(inst, relaxed = True)
    
    # Computar solução ótima
    model.optimize()
    
    # Relatar outros dados
    print(
        '\nTempo de execução: ' + 
        str(model.runTime) + 
        '; Número de variáveis: ' + 
        str(model.numVars) + 
        '\n'
    )



SOL. DA INSTÂNCIA COM 50 CLIENTES:

Using license file /home/eduinnarelli/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 1800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 2122 rows, 36736 columns and 101024 nonzeros
Model fingerprint: 0x9e691a63
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.06s
Presolved: 2122 rows, 36736 columns, 101024 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.649e+04
 Factor NZ  : 2.226e+05 (roughly 17 MBytes of memory)
 Factor Ops : 5.236e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual  